### Limpieza de datos tweets en español

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import re
import string
import seaborn as sns

In [2]:
#Carga de archivo
df =  pd.read_csv('twitter.txt', sep=">")

In [3]:

df['User - Location'] = df['User - Location'].astype(str)

# Limpieza de datos

Para la clasificación se necesitara limpiar un poco las columnas de tweets

In [4]:
#Eliminar URL's, nombres de usuarios 

isURL = re.compile(r'http[s]?:// (?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', re.VERBOSE | re.IGNORECASE)
isRTusername = re.compile(r'^RT+[\s]+(@[\w_]+:)',re.VERBOSE | re.IGNORECASE) #r'^RT+[\s]+(@[\w_]+:)'
isEntity = re.compile(r'@[\w_]+', re.VERBOSE | re.IGNORECASE) 

def clean_tweet(row):
    row = isURL.sub("",row)
    row = isRTusername.sub("",row)
    row = isEntity.sub("",row)
    return row

df['Tweet'] = df['Tweet'].apply(lambda row:clean_tweet(row))



# Eliminamos salto de linea

def Sep_a(city):
    return city.replace('\n'," ").replace('\r'," ")  

df['Tweet'] = df['Tweet'].apply(lambda x: Sep_a(x))



# Limpiando signos de puntuación,emoticones
def remove_punctuation ( text ):
    return re.sub(r'[^A-Za-z0-9\s]', "", text)

def normalize(city):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
        ("ü", "u"),
    )
    for a, b in replacements:
        city = city.replace(a, b).replace(a.upper(), b.upper()).capitalize().strip()
    return  city



df['Tweet'] = df['Tweet'].apply(lambda x: normalize(x))
df['Tweet'] = df['Tweet'].apply(lambda x: remove_punctuation(x))
    

# Cambiar a minusculas todas las palabras

df['Tweet'] = df['Tweet'].apply(lambda x: x.lower())


# Limpiar Ciudades

In [5]:
def Sep_c(city):
    city = city.split(',')
    return city[0]

def Sep_h(city):
    city = city.split('-')
    return city[0]

def Sep_a(city):
    return city.replace('\n'," ").replace('\r'," ").replace('\t'," ")

def Sep_e(city):
    return city.replace('  '," ")

#Corregir con errores de tipeo

def Medellin(city):
    return  re.sub(r'(?is)Medell.+', 'Medellin', city)

def Bogota(city):
    return  re.sub(r'(?is)Bog.+', 'Bogota', city)

def Popayan(city):
    return  re.sub(r'(?is)Popay.+', 'Popayan', city)
def Ibague(city):
    return city.replace('Colombia Ibague Tolima',"Ibague")




In [6]:
df['User - Location'] = df['User - Location'].apply(lambda x : normalize(x))
df['User - Location'] = df['User - Location'].apply(lambda x : Sep_c(x))
df['User - Location'] = df['User - Location'].apply(lambda x : Sep_h(x))
df['User - Location'] = df['User - Location'].apply(lambda x : Sep_e(x))
df['User - Location'] = df['User - Location'].apply(lambda x: remove_punctuation(x))
df['User - Location'] = df['User - Location'].apply(lambda x : Sep_a(x))
df['User - Location'] = df['User - Location'].apply(lambda x : Bogota(x))
df['User - Location'] = df['User - Location'].apply(lambda x : Medellin(x))
df['User - Location'] = df['User - Location'].apply(lambda x : Popayan(x))
df['User - Location'] = df['User - Location'].apply(lambda x : Ibague(x))

In [7]:
# Eliminanando colomnas no necesarias
df.drop((['User', 'User - Name', 'User - Description', 'User - Time Zone']),axis = 1, inplace = True)
df.dropna(subset= ["Tweet"],inplace = True)

In [8]:
# Agregando Columna Regiones

# 1. Función de clasificación

def regiones(city):
    rgs = {'Bogota': 'Cundinamarca','Medellin':'Antioquia','Envigado':'Antioquia', 'Cali':'Valle del Cauca',
           'Barranquilla':'Atlantico', 'Cartagena':'Bolivar', 'Cucuta':'Norte de Santander', 'Villavicencio':'Meta',
           'Ibague':'Tolima', 'Bucaramanga':'Santander', 'Popayan':'Cauca', 'Pasto':'Nariño'}
    
    if city in rgs.keys():
        region = rgs.get(city)
    else:
        region = city
    return region

# 2. Aplicando lambda
df['Regiones'] = df['User - Location'].apply(lambda x : regiones(x))

In [9]:
# Eliminar filas con valores Nan en Columna Tweet
df = df.dropna(subset= ['Tweet'])


In [10]:
# cargando predicciones
Predicts = pd.read_csv('Predicts.csv')
df['Sentimiento'] = Predicts['Sentimiento']
#Guardar datos para usar
df.to_csv('Data_clean.csv',index = False, encoding = 'utf-8' )


# Crear datos para la nube de palabras

In [11]:
stop_words = ['a','acá','ahí','al', 'algo','ambos','ante','antes','aquel','aquí','así','aun','aunque','bajo','bastante','cabe','cada','casi','como','con','cual','cuales','cuan','cuando',
              'de','dejar','del','desde','donde','dos','el','él','ella','ello','en','encima','entonces','entre','era', 'eramos','eran','eres','es','esa', 'eso', 'eso', 'esas','esos',
              'esta', 'estas', 'estos','esto','este','etc','ha','hace', 'haces','junto', 'juntos','la', 'las', 'los', 'lo','más','me','menos','mi','mis','mia', 'mias','mio','mios','misma',
              'mismo','ni','os','otra','otro', 'otras','otros','para','pero','por','por qué','porque','primero','pues','que','qué','quizas', 'quiza','se','sr','sra','sres','sta','su', 'sus',
              'tan','te','tú','tu','tus','tuya','tuyo', 'tuyas','tuyos','un', 'una', 'unos', 'unas','y', 'mas', 'si', 'le', 'e', 'do', 'son', 'no','1', '2t','nos','001', 'da']

def stop_w(row):
    
    list_row = row.split()
    for i in stop_words:
        for j in range(len(list_row)-1, -1, -1):
            if list_row[j] == i:
                    del(list_row[j])
    return " ".join(list_row)
            
df['tweet_SW'] = df['Tweet'].apply(lambda x:stop_w(x))   

In [12]:
New_positive = df[df['Sentimiento'] == 'P']
New_negative = df[df['Sentimiento'] == 'N']
New_neutral = df[df['Sentimiento'] == 'NEU']


In [13]:
# Funcion para convertir todas las palabras en string
def list_words(column):
    list_ = []
    r = column.shape[0]
    for i in range(0,r):
        text = column.iloc[i].split()
        for j in text:
            list_.append(j)
    return " ".join(list_)
                




In [15]:
#String de palabras filtradas por sentimiento

t_positive = list_words(New_positive['tweet_SW'])
t_negative = list_words(New_negative['tweet_SW'])
t_neutral = list_words(New_neutral['tweet_SW'])

# Generar nube de palabras

In [16]:
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator

# Cambiar a fondo blanco imagen

def transform_white_backgroud(png_path):
    picture = Image.open(png_path).convert("RGBA")
    image = Image.new("RGB", picture.size, "WHITE")
    image.paste(picture, (0, 0), picture)
    mask = np.array(image)
    return mask


In [18]:
# Generando imagenes

# 1. Colocar fondo blanco a la imagen
mask = transform_white_backgroud("Colombia.png")


# 2. Generar imagenes 
word_cloud_positive = WordCloud(mask=mask, background_color='rgb(38,50,56)', contour_width=1, contour_color='grey', max_words=200, min_font_size=5, collocation_threshold=10,colormap='Pastel1').generate(t_positive)
word_cloud_positive.to_file("positive_words.png")

word_cloud_negative = WordCloud(mask=mask, background_color='rgb(38,50,56)', contour_width=1, contour_color='grey', max_words=200, min_font_size=5, collocation_threshold=10,colormap='Pastel1').generate(t_negative)
word_cloud_negative.to_file("negative_words.png")

word_cloud_neutral = WordCloud(mask=mask, background_color='white', contour_width=1, contour_color='grey', max_words=200, min_font_size=5, collocation_threshold=10).generate(t_neutral)
word_cloud_neutral .to_file("neutral_words.png")